In [17]:
from typing import Tuple
import tensorflow as tf
from load_fer2013 import load_fer2013, preprocess

def load_and_preprocess_data() -> Tuple[tf.data.Dataset, tf.data.Dataset, tf.data.Dataset]:
    data = load_fer2013()
    num_classes = 7

    # Define splits for train, validation, and test sets
    split_train = int(len(data) * 0.7)
    split_test = int(len(data) * 0.1)
    split_val = len(data) - split_train - split_test

    # Create a TensorFlow dataset from the data
    dataset = tf.data.Dataset.from_tensor_slices(dict(data))
    dataset = dataset.map(
        lambda row: preprocess(row, num_classes), num_parallel_calls=tf.data.AUTOTUNE
    )

    # Partition the data into train, validation, and test sets
    train_dataset = (
        dataset.take(split_train).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
    )
    val_dataset = (
        dataset.skip(split_train).take(split_val).batch(32).prefetch(tf.data.AUTOTUNE)
    )
    test_dataset = (
        dataset.skip(split_train + split_val).batch(32).prefetch(tf.data.AUTOTUNE)
    )

    return train_dataset, val_dataset, test_dataset

In [20]:
import tensorflow as tf
from keras.models import load_model
import numpy as np

# Load the saved model from the specified path
model_path = 'output/best_model'
model = load_model(model_path)

train_dataset, val_dataset, test_dataset = load_and_preprocess_data()

# metrics from model.evaluate
val_accuracy = model.evaluate(test_dataset)

print(f"Testing accuracy: {val_accuracy}")

# Get predictions for test data
predictions = model.predict(test_dataset)

# Since 'predictions' is a 2D array, each row corresponds to predictions for a given input
# To get the first prediction, we select the first row
first_prediction = predictions[0]

# Get the class with the highest probability from the first prediction
predicted_class = np.argmax(first_prediction)
print(f"First prediction {first_prediction}")
print(f"Predicted class for the first test example: {predicted_class} = Happy")

Loading dataset...
113/113 [==============================] - 3s 13ms/step - loss: 1.2757 - accuracy: 0.5256 - categorical_accuracy: 0.5256
Testing accuracy: [1.275729775428772, 0.5256410241127014, 0.5256410241127014]
113/113 [==============================] - 3s 13ms/step
First prediction [0.32203916 0.01416158 0.04879333 0.24709927 0.19526306 0.02899002
 0.14365356]
Predicted class for the first test example: 0 = Happy
